In [8]:
import pinocchio as pin
import numpy as np
import time
from numpy.linalg import pinv,inv,norm,svd,eig
from utils.tiago_loader import loadTiago
import matplotlib.pylab as plt; plt.ion()
from utils.meshcat_viewer_wrapper import MeshcatVisualizer
from typing import List, Union, Optional, Tuple

from inverse_kinematics_updates import get_config_velocity_update_translation_with_proj
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load an extra cube in the viewer to figure a table. First, control the robot hand to reach an arbitrary point on the table (don't worry about collisions). Then, implement a control law to control three tasks:
* The tool frame should be kept on the table = **vertical** constraint  (only the vertical component of the error matters, select the third row of the Jacobian and error accordingly).
* The gaze should reach the position of a ball positionned on the table.
* The center of the mobile base frame should reach a given goal on the floor. For this task, only the horizontal components (x- and y-) of the task matter, select only the first two rows of the Jacobian and error accordingly.

In [2]:
robot = loadTiago(addGazeFrame=True)
viz = MeshcatVisualizer(robot)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [3]:
TABLE_HEIGHT = 2.

In [5]:
IDX_GAZE = robot.model.getFrameId('framegaze')
IDX_TOOL = robot.model.getFrameId('frametool')
IDX_BASIS = robot.model.getFrameId('framebasis')

# Add a small ball as a visual target to be reached by the robot
# ball = np.array([ 1.2,0.5,1.1 ])
ball = np.array([ 1.2,0.5,TABLE_HEIGHT + 0.2])
o_Mgazegoal = pin.SE3(np.eye(3), ball)
viz.addSphere('ball', .05, [ .8,.1,.5, .8] )
viz.applyConfiguration('ball', list(ball)+[0,0,0,1])

# Add the target on the floor
# basis_target_coordinates = ball + np.array([ 0.5 , 0.2, -ball[-1]])
basis_target_coordinates = ball + np.array([-0.2 , -0.2, -ball[-1]])
o_M_floorTarget = pin.SE3(np.eye(3), basis_target_coordinates)
viz.delete('basis_target')
viz.addBox('basis_target', [.1,.1,.4], [ 1.,0.,0., 1.] )
viz.applyConfiguration('basis_target',o_M_floorTarget)


# Add the table
table_coordinates = np.array([ 1., 1., TABLE_HEIGHT])
# table_coordinates = np.array([ -1.5, 1., TABLE_HEIGHT])
o_Mtable = pin.SE3(np.eye(3), table_coordinates)
viz.delete('table')
viz.addBox('table', [2. , 2.,.01], [ 1., 1., 1., .5] )
viz.applyConfiguration('table',o_Mtable)

# Robot initial configuration.
q0 = np.array([ 0.  ,  0.  ,  1.  ,  0.  ,  0.18,  1.37, -0.24, -0.98,  0.98,
                0.  ,  0.  ,  0.  ,  0.  , -0.13,  0.  ,  0.  ,  0.  ,  0.  ])
viz.display(q0)
pin.framesForwardKinematics(robot.model,robot.data,q0)
# Then get the position
o_Mtool_start = robot.data.oMf[IDX_TOOL]
viz.delete("start")
viz.addBox('start', [.1,.1,.1], [ 1.,.1,.5, .6] )
viz.applyConfiguration('start',o_Mtool_start)


In [6]:
def reset(viz):
    viz.display(q0)
    for i in range(2000): 
        viz.delete(f"debug/traj{i}")
        viz.delete(f"debug/traj_gaze{i}")
    viz.delete("current_6D")
    viz.delete("gaze")
reset(viz)

### Simple testing the position (1/2/3D) constraint
```python
# TOOL STAYS ON THE TABLE
vq = get_config_velocity_update_translation(q, robot, IDX_TOOL, o_Mtable, constraint=(2,3))

# TOOL REACHES THE CENTER OF THE TABLE but not Z
vq = get_config_velocity_update_translation(q, robot, IDX_TOOL, o_Mtable, constraints=(0,2))

# BASIS REACHES 2D XY TARGET
vq = get_config_velocity_update_translation(q, robot, IDX_BASIS, o_M_floorTarget, constraint=(0,2))
```

$ P_1 = I_{nq} - J_1^+ J_1 $

> copyright: someone helped!

$ P_2 = P_1 - {(J_2 P_1)}^+ {J_2 P_1} $

In [13]:
# reset(viz)

def solve_tasks(
    task_list: List[Tuple[int, pin.SE3, Tuple[int, int]]],
    DT:float=3e-2,
    Niter:int=700,
    viz=None,
    q_init=q0,
    rob=robot
):
    q = q_init.copy()
    viz.display(q_init)

    for i in range(Niter):  # Integrate over 2 second of rob life
        pin.framesForwardKinematics(rob.model, rob.data, q)  # update Forward kinematic
        vq, p = None, None

        for frame_id, o_M_target, constraints in task_list:
            vq, p = get_config_velocity_update_translation_with_proj(
                q, rob, frame_id, o_M_target, constraints=constraints,
                vq_prev=vq, projector=p
            )
        
        q = pin.integrate(rob.model, q, vq*DT)
        viz.display(q)
        time.sleep(1.E-3)


In [14]:
# TOOL ON THE TABLE Z=[2:3]
TASK_1 = (IDX_TOOL, o_Mtable, (2,3))
# BASIS TOWARD THE CHECKPOINT ON THE FLOOR XY =[0:2]
TASK_2 = (IDX_BASIS, o_M_floorTarget, (0,2))
# GAZE TOWARD THE BALL XYZ =[0:3]
TASK_3 = (IDX_GAZE, o_Mgazegoal, (0,3))
FULL_TASKS = [TASK_1, TASK_2, TASK_3]


In [15]:
solve_tasks([TASK_1], DT=3e-2, Niter=700, viz=viz, q_init=q0, rob=robot)

------------------------

Task 1 ONLY